In [2]:
import os,shutil
import cv2,keras
import numpy as np
from imgaug import augmenters as iaa
import skimage.io as io
from sklearn.model_selection import train_test_split
from tensorflow.python.keras.applications import ResNet50
from keras.optimizers import Adam

In [3]:
def create_dataset(path,num_classes,data_splitno):
    data_split=[]
    subjects = os.listdir(path) # getting each subject folders (no. of classes=166)
    for subject in subjects[:num_classes]:     ## selcting only 150 classes
        os.makedirs(os.path.join('Data/train',subject))  ## creating new folder for train and test with 60/40 split from each subjects
        os.makedirs(os.path.join('Data/test',subject))
        subject_images = os.listdir(os.path.join(path,subject))
        for image in subject_images:
            if(image.endswith('.png')):
                data_split.append(image)
                train_no = int(data_splitno*len(data_split))
        for i in range(0,len(data_split)):
        
            if(i<=train_no-1):
                source_train=os.path.join(path,subject,data_split[i])
                destination_train=os.path.join('Data/train',subject)
                shutil.copy(source_train,destination_train)
        
            else:
                source_test=os.path.join(path,subject,data_split[i])
                destination_test=os.path.join('Data/test',subject)
                shutil.copy(source_test,destination_test)
                
        data_split=[] 

In [4]:
def data_augmentation(image,path):
    flip =iaa.Sequential([iaa.Fliplr(0.5)]).augment_images(image)
    io.imsave(path+'flip.png',flip)
    
    crop = iaa.Sequential([iaa.Crop(percent=(0,0.01))]).augment_images(image)
    io.imsave(path+'crop.png',crop)
    
    GB = iaa.Sequential([iaa.GaussianBlur(sigma=(0,3.0))]).augment_images(image)
    io.imsave(path+'GB.png',GB)
    
    GN = iaa.Sequential([iaa.AdditiveGaussianNoise(scale=(0.0,0.2))]).augment_images(image)
    io.imsave(path+'GN.png',GN)
    
    CN = iaa.Sequential([iaa.ContrastNormalization(0.5,per_channel=0.5)]).augment_images(image)
    io.imsave(path+'CN.png',CN)
    
    BR = iaa.Sequential([iaa.Multiply((0.8,1.2))]).augment_images(image)
    io.imsave(path+'BR.png',BR)
    
    scale= iaa.Sequential([iaa.Affine(scale={"x": (0.8, 1.2), "y": (0.8,1.2)})]).augment_images(image)
    io.imsave(path+'scale.png',scale)
    
    rotate = iaa.Sequential([iaa.Affine(rotate=(-45,45))]).augment_images(image)
    io.imsave(path+'rotate.png',rotate)
    

In [5]:
def get_augmented_data(path):
    subjects = os.listdir(path)
    for subject in subjects:
        subject_path=os.path.join(path,subject)
        subject_images = os.listdir(subject_path)
        for image in subject_images:
            if(image.endswith('.png')):
                image_path=os.path.join(path,subject,image)
                image = cv2.resize(cv2.imread(image_path),(100,100))
                im_path=os.path.join(os.path.splitext(image_path)[0])
                data_augmentation(image,im_path)

In [ ]:
def data(path):
    images=[]
    label=[]
    subjects = os.listdir(path)
    for subject in subjects:
        subject_path=os.path.join(path,subject)
        subject_images = os.listdir(subject_path)
        for image in subject_images:
            if(image.endswith('.png')):
                image_path=os.path.join(path,subject,image)
                image = cv2.resize(cv2.imread(image_path),(224,224))
                images.append(image)
                label.append(subject)
    
    images=np.array(images)
    label=np.array(label)
    return images,label

In [6]:
path = 'Dataset/Train Dataset'
#train = 'Train Dataset'
num_classes=150
data_splitno=0.6
create_dataset(path,num_classes,data_splitno)

# Data Augmentation
train_path = 'Data/train'
test_path = 'Data/test'
get_augmented_data(train_path)
get_augmented_data(test_path)

#Getting train and test data(80/20)
train_images,train_label= data(train_path)
test_images,test_label= data(test_path)
x_train,x_valid,y_train,y_valid=train_test_split(train_images,train_label,test_size=0.2,random_state=42)
x_test=test_images
y_test=test_label

# convert class vectors to binary class matrices
y_train=keras.utils.to_categorical(y_train,1000)
y_valid= keras.utils.to_categorical(y_valid,1000)
y_test= keras.utils.to_categorical(y_test,1000)
print('x_train=',x_train.shape)
print('y_train=',y_train.shape)
print('x_test=',x_test.shape)
print('y_test=',y_test.shape)

C:\Users\Vyshnav\Anaconda3\lib\site-packages\imgaug\augmenters\meta.py:464: UserWarning: You provided a numpy array of shape (100, 100, 3) as input to augment_images(), which was interpreted as (N, H, W). The last dimension however has value 1 or 3, which indicates that you provided a single image with shape (H, W, C) instead. If that is the case, you should use augment_image(image) or augment_images([image]), otherwise you will not get the expected augmentations.
  "you will not get the expected augmentations." % (images_copy.shape,))
C:\Users\Vyshnav\Anaconda3\lib\site-packages\skimage\io\_io.py:140: UserWarning: Data/train\015\03CN.png is a low contrast image
  warn('%s is a low contrast image' % fname)
C:\Users\Vyshnav\Anaconda3\lib\site-packages\skimage\io\_io.py:140: UserWarning: Data/train\041\04CN.png is a low contrast image
  warn('%s is a low contrast image' % fname)
C:\Users\Vyshnav\Anaconda3\lib\site-packages\skimage\io\_io.py:140: UserWarning: Data/train\041\04rotate.png i

NameError: name 'data' is not defined

In [ ]:
model = ResNet50(include_top=True, weights=None, input_tensor=None, input_shape=x_train[0].shape, 
                 pooling=None, classes=1000)
model.summary()

In [ ]:
model.compile(loss='categorical_crossentropy',optimizer=Adam(),metrics=['accuracy'])
model.fit(x_train, y_train,batch_size=70,epochs=1,verbose=1,validation_data=(x_valid, y_valid))


In [ ]:
score = model.evaluate(x_test, y_test, verbose=0)
print('Test loss:', score[0])
print('Test accuracy:', score[1])

In [17]:
# ##Dataset creation
# subjects = os.listdir(os.path.join(path,train)) # getting each subject folders (no. of classes=166)
# for subject in subjects[:num_classes]:     ## selcting only 150 classes
#     os.makedirs(os.path.join('Data/train',subject))  ## creating new folder for train and test with 60/40 split from each subjects
#     os.makedirs(os.path.join('Data/test',subject))
#     subject_images = os.listdir(os.path.join(path,train,subject))
#     for image in subject_images:
#         if(image.endswith('.png')):
#             data_split.append(image)
#             train_no = int(data_splitno*len(data_split))
#     for i in range(0,len(data_split)):
        
#         if(i<=train_no-1):
#             source_train=os.path.join(path,train,subject,data_split[i])
#             destination_train=os.path.join('Data/train',subject)
#             shutil.copy(source_train,destination_train)
        
#         else:
#             source_test=os.path.join(path,train,subject,data_split[i])
#             destination_test=os.path.join('Data/test',subject)
#             shutil.copy(source_test,destination_test)
     
#     data_split=[]  

In [21]:
# def data_augmentation(image,path):
#     flip =iaa.Sequential([iaa.Fliplr(0.5)]).augment_images(image)
#     io.imsave(path+'flip.png',flip)
    
#     crop = iaa.Sequential([iaa.Crop(percent=(0,0.01))]).augment_images(image)
#     io.imsave(path+'crop.png',crop)
    
#     GB = iaa.Sequential([iaa.GaussianBlur(sigma=(0,3.0))]).augment_images(image)
#     io.imsave(path+'GB.png',GB)
    
#     GN = iaa.Sequential([iaa.AdditiveGaussianNoise(scale=(0.0,0.2))]).augment_images(image)
#     io.imsave(path+'GN.png',GN)
    
#     CN = iaa.Sequential([iaa.ContrastNormalization(0.5,per_channel=0.5)]).augment_images(image)
#     io.imsave(path+'CN.png',CN)
    
#     BR = iaa.Sequential([iaa.Multiply((0.8,1.2))]).augment_images(image)
#     io.imsave(path+'BR.png',BR)
    
#     scale= iaa.Sequential([iaa.Affine(scale={"x": (0.8, 1.2), "y": (0.8,1.2)})]).augment_images(image)
#     io.imsave(path+'scale.png',scale)
    
#     rotate = iaa.Sequential([iaa.Affine(rotate=(-45,45))]).augment_images(image)
#     io.imsave(path+'rotate.png',rotate)
    